## AIMI High School Internship 2023
### Notebook 2: Training a Vision Model to Predict ET Distances

**The Problem**: Given a chest X-ray, our goal in this project is to predict the distance from an endotracheal tube to the carina. This is an important clinical task - endotracheal tubes that are positioned too far (>5cm) above the carina will not work effectively.

**Your Second Task**: You should now have a training dataset consisting of (a) chest X-rays and (b) annotations indicating the distance of the endotracheal tube from the carina. Now, your goal is to train a computer vision model to predict endotracheal tube distance from the image. You have **two options** for this task, and you may attempt one or both of these:
- *Distance Categorization* : Train a model to determine whether the position of a tube is abnormal (>5.0 cm) or normal (≤ 5.0 cm).
- *Distance Prediction*: Train a model that predicts the distance of the endotracheal tube from the carina in centimeters.

In this notebook, we provide some simple starter code to get you started on training a computer vision model. You are not required to use this template - feel free to modify as you see fit.

**Submitting Your Model**: We have created a leaderboard where you can submit your model and view results on the held-out test set. We provide instructions below for submitting your model to the leaderboard. **Please follow these directions carefully**.

We will evaluate your results on the held-out test set with the following evaluation metrics:
- *Distance Categorization* : We will measure AUROC, which is a metric commonly used in healthcare tasks. See this blog for a good explanation of AUROC: https://glassboxmedicine.com/2019/02/23/measuring-performance-auc-auroc/
- *Distance Prediction*: We will measure the mean average error (also known as L1 distance) between the predicted distances and the true distances.


## Load Data
Before you begin, make sure to go to `Runtime` > `Change Runtime Type` and select a T4 GPU. Then, upload `data.zip`. It should take about 10 minutes for these files to be uploaded. Then, run the following cells to unzip the dataset (which should take < 10 seconds)

## Import Libraries
We are leveraging the PyTorch framework to train our models. For more information and tutorials on PyTorch, see this link: https://pytorch.org/tutorials/beginner/basics/intro.html

In [19]:
# Some libraries that you may find useful are included here.
# To import a library that isn't provided with Colab, use the following command: !pip install torchmetrics
import torch
import pandas as pd
from PIL import Image
import numpy as np
from tqdm import tqdm
from torchvision import transforms
from torch.utils.data import DataLoader, random_split


## Create Dataloaders
We will implement a custom Dataset class to load in data. A custom Dataset class must have three methods: `__init__`, which sets up any class variables, `__len__`, which defines the total number of images, and `__getitem__`, which returns a single image and its paired label.

In [20]:
from torch.utils.data import Dataset
from PIL import Image

class ChestXRayDataset(Dataset):
    def __init__(self, img_paths, labels, distances):
        super(ChestXRayDataset, self).__init__()
        self.img_paths = img_paths
        self.labels = labels
        self.distances = distances
        # Fill in __init__() here

    def __len__(self):

        # Fill in __len__() here
        return self.labels.shape[0]

    def __getitem__(self, idx):
        out_dict = {"idx": torch.tensor(idx),}

        # Fill in __getitem__() here
        img = Image.open(f"data/{self.img_paths[idx]}")
        convert_tensor = transforms.Compose([
            transforms.Resize(size=224),
            transforms.ToTensor(),          
        ])
        img_as_tensor = convert_tensor(img)
        img_as_tensor.requires_grad_ = True
        out_dict["img"] = img_as_tensor
        out_dict["labels"] = self.labels[idx]
        out_dict["distance"] = self.distances[idx]

        return out_dict

## Define Training Components
Here, define any necessary components that you need to train your model, such as the model architecture, the loss function, and the optimizer.

In [21]:
import torch.nn as nn

# loss = nn.CrossEntropyLoss()
# model = # Model Architecture (make sure to load the model on GPU, not CPU!)
# opt = torch.optim.AdamW(model.parameters(), lr=1e-4) # AdamW is a commonly-used optimizer. Feel free to modify.

data = pd.read_csv("mimic_train_labels_pruned.csv")
img_paths = data["image_path"].to_numpy()
labels = data["positioning"].to_numpy()
distances = data["measures"].to_numpy()

dataset = ChestXRayDataset(img_paths=img_paths, labels=labels, distances=distances)

def get_train_val_split(dataset, batch_size=10, train_prop=0.8):
    dataset_length = len(dataset)
    train_length = int(dataset_length * train_prop)
    val_length = dataset_length - train_length
    train_dataset, val_dataset = random_split(
            dataset, [train_length, val_length]
        )

    train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    return train_loader, val_loader

train_loader, val_loader = get_train_val_split(dataset, batch_size=32)

print(len(train_loader.dataset))
print(len(val_loader.dataset))


9252
2313


## Training Code
We provide starter code below that implements a simple training loop in PyTorch. Feel free to modify as you see fit.

In [22]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

def calculate_scores(y_true, y_pred):
   return f1_score(y_true, y_pred), roc_auc_score(y_true, y_pred), accuracy_score(y_true, y_pred) 

def validate(model, loss_fn, val_loader):
    
    f1_scores, rocauc_scores, acc_scores = [], [], []
    total_loss = 0
    for data in tqdm(val_loader):
        model.eval()
        with torch.no_grad():
            inputs = data["img"]
            labels = data["labels"]
            outputs = model(inputs)
            _, preds = torch.max(outputs, dim=1)
            loss_val = loss_fn(preds, labels)
            total_loss += loss_val.item()
            loss_val.backward()

        f1, rocauc, acc = calculate_scores(labels.detach().cpu().numpy(), preds.detach().cpu().numpy())
        f1_scores.append(f1)
        rocauc_scores.append(rocauc)
        acc_scores.append(acc)
    return np.mean(f1_scores), np.mean(rocauc_scores), np.mean(acc_scores)

def train(model, loss_fn, train_loader, opt):
    f1_scores, rocauc_scores, acc_scores = [], [], []
    total_loss = 0
    for data in tqdm(train_loader):
        model.train()
        inputs = data["img"]
        labels = data["labels"]
        opt.zero_grad()
        print(inputs.shape)
        outputs = model(inputs)
        _, preds = torch.max(outputs, dim=1)
        loss_val = loss_fn(preds, labels)
        total_loss += loss_val.item()
        loss_val.backward()
        opt.step()

        f1, rocauc, acc = calculate_scores(labels.detach().cpu().numpy(), preds.detach().cpu().numpy())
        f1_scores.append(f1)
        rocauc_scores.append(rocauc)
        acc_scores.append(acc)
    return np.mean(f1_scores), np.mean(rocauc_scores), np.mean(acc_scores), total_loss

def batch_progress(epoch, tr_f1, tr_acc, tr_roc, tr_loss, val_f1, val_acc, val_roc, val_loss):
    # Batch train data
    print(f"Epoch {epoch} Training Statistics")
    print(f"F1 Score: {tr_f1}\n Accuracy: {tr_acc}\n AUROC: {tr_roc}\n Loss: {tr_loss}\n")
    # Batch validation data
    print(f"Epoch {epoch} Validation Statistics")
    print(f"F1 Score: {val_f1}\n Accuracy: {val_acc}\n AUROC: {val_roc}\n Loss: {val_loss}\n")



In [23]:
# Model definition
import torch.nn as nn
# Load resnet-50 here

# FineTuning Architecture
# From https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8933872/#b50

from torchvision import datasets, transforms, models

def get_model(num_classes):
    resnet50_aimi = models.resnet50(pretrained=True)
    n_features = resnet50_aimi.fc.in_features
    try:
        # model.fc = nn.Linear(n_features, K)
        resnet50_aimi.fc = nn.Sequential(
            nn.Linear(n_features, n_features),
            nn.Dropout(0.5),
            nn.Linear(n_features, n_features),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(n_features, num_classes)
        )
    except Exception as e:
        print("ERROR at: model.fc = nn.Linear(n_features, K)")
        raise e
    return resnet50_aimi

model = get_model(num_classes=2)
print(model)


/opt/miniconda3/envs/pyro/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/miniconda3/envs/pyro/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [24]:
import torch.nn as nn
import numpy as np

opt = torch.optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999))
loss_fn = nn.CrossEntropyLoss()

NUM_EPOCHS = 10

train_f1, train_rocauc, train_acc, train_loss = [], [], [], []
val_f1, val_rocauc, val_acc, val_loss = [], [], [], []
for epoch in range(NUM_EPOCHS):
    batch_tr_f1, batch_tr_rocauc, batch_tr_acc, batch_tr_loss = train(model, loss_fn, train_loader, opt)
    batch_val_f1, batch_val_rocauc, batch_val_acc, batch_val_loss = validate(model, loss_fn, val_loader)

    batch_progress(
        batch_tr_f1, batch_tr_acc, batch_tr_rocauc, batch_tr_loss, 
        batch_val_f1, batch_val_acc, batch_tr_rocauc, batch_val_loss
        )
    
    train_f1.append(batch_tr_f1)
    train_rocauc.append(batch_tr_rocauc)
    train_acc.append(batch_tr_acc)
    train_loss.append(batch_tr_loss)
    
    val_f1.append(batch_val_f1)
    val_rocauc.append(batch_val_rocauc)
    val_acc.append(batch_val_acc)
    train_loss.append(batch_val_loss)

train_f1 = np.array(train_f1)
train_rocauc = np.array(train_rocauc)
train_acc = np.array(train_acc)
train_loss = np.array(train_loss)

val_f1 = np.array(val_f1)
val_rocauc = np.array(val_rocauc)
val_acc = np.array(val_acc)
val_loss = np.array(val_loss)

  0%|          | 0/289 [00:00<?, ?it/s]

torch.Size([32, 1, 224, 224])


RuntimeError: Given groups=1, weight of size [64, 3, 7, 7], expected input[32, 1, 224, 224] to have 3 channels, but got 1 channels instead

## Submitting Your Results
Once you have successfully trained your model, generate predictions on the test set and save your results as a `.csv` file. This file can then be uploaded to the leaderboard.

Your final `.csv` file **must** have the following format:
- There must be a column titled `image_path` with the paths to the test set images. This column should be identical to the one provided in `mimic_test_student.csv`.
- There must be a column titled `pred` with your model outputs.
  - If you are running the `distance categorization` task, this column must have floating point numbers ranging between 0 and 1. Higher numbers should indicate a greater likelihood that the tube distance is abnormal. Hint: You can convert model outputs to the 0 to 1 range by applying the sigmoid activation function (torch.nn.sigmoid())
  - If you are running the `distance prediction` task, this column must have numbers representing the tube distance in centimeters.
- Double check that there are 500 rows in your output file

In [ ]:
model = # Model Architecture
ckpt = torch.load("/content/best.pkl")
model.load_state_dict(ckpt["state_dict"])

test_dataset = ChestXRayDataset("""Fill in args here""")
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=4, shuffle=False, drop_last=False)

test_results = {"image_path": [], "pred": []}
# Write method to load in data from test_loader, compute model predictions, and append results to test_results dict


In [ ]:
test_results = pd.DataFrame(test_results)
test_results.to_csv(f"/content/test.csv")